In [15]:
import pandas as pd
import numpy as np
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# --- Configuration ---
ASTHMA_DATA_PATH = "/content/drive/My Drive/Big Data Project/Data/Raw/asthma_dataset/Asthma Prevalance_Data_2020_2023.csv"
POLLUTANT_DATA_PATH = "/content/drive/My Drive/Big Data Project/Data/Processed/all_pollutants_merged_inner.csv"

# Define pollutants to process
POLLUTANTS = ['PM25', 'O3', 'NO2', 'SO2', 'CO']

# Meaningful thresholds values relevant to health guidelines
THRESHOLDS = {
    'PM25': 9.0,  # EPA AQI "Moderate" 24-hour PM2.5 (ug/m3)
    'O3': 0.070,   # EPA AQI "Moderate" 8-hour Ozone (ppm)
    'NO2': 53,    # EPA 1-hour NO2 standard (ppb)
    'SO2': 0.1,     # EPA 1-hour SO2 standard (ppb)
    'CO': 0.35       # EPA 8-hour CO standard (ppm)
}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# --- 1. Load Data ---
print("--- 1. Loading Data ---")
try:
    df_asthma = pd.read_csv(ASTHMA_DATA_PATH)
    print(f"Asthma data loaded: {df_asthma.shape}")
except FileNotFoundError:
    print(f"ERROR: Asthma data file not found at {ASTHMA_DATA_PATH}")
    df_asthma = pd.DataFrame() # Create empty to avoid downstream errors if file not found

try:
    df_pollutant_raw = pd.read_csv(POLLUTANT_DATA_PATH, low_memory=False)
    print(f"Pollutant data loaded: {df_pollutant_raw.shape}")
except FileNotFoundError:
    print(f"ERROR: Pollutant data file not found at {POLLUTANT_DATA_PATH}")
    print("Please ensure the POLLUTANT_DATA_PATH is correct and the file exists.")
    df_pollutant_raw = pd.DataFrame() # Create empty

--- 1. Loading Data ---
Asthma data loaded: (224, 4)
Pollutant data loaded: (98351, 11)


In [17]:
# --- 2. Prepare Pollutant Data ---
if not df_pollutant_raw.empty:
    print("\n--- 2. Preparing Pollutant Data ---")
    df_pollutant = df_pollutant_raw.copy()

    # Convert 'Date Local' to datetime
    if 'Date Local' in df_pollutant.columns:
        df_pollutant['Date Local'] = pd.to_datetime(df_pollutant['Date Local'], errors='coerce')
        df_pollutant.dropna(subset=['Date Local'], inplace=True) # Drop rows where date conversion failed
        df_pollutant['Year'] = df_pollutant['Date Local'].dt.year
        df_pollutant['Month'] = df_pollutant['Date Local'].dt.month
    else:
        print("ERROR: 'Date Local' column not found in pollutant data. Cannot proceed with date-based features.")
        df_pollutant = pd.DataFrame() # Stop processing if no date

    # Standardize 'County Name'
    if 'County Name' in df_pollutant.columns:
        df_pollutant['County Name'] = df_pollutant['County Name'].astype(str).str.strip().str.lower()
    else:
        print("ERROR: 'County Name' column not found in pollutant data. Cannot group by county.")
        df_pollutant = pd.DataFrame()

    # Ensure pollutant columns are numeric
    for p in POLLUTANTS:
        if p in df_pollutant.columns:
            df_pollutant[p] = pd.to_numeric(df_pollutant[p], errors='coerce')
        else:
            print(f"Warning: Pollutant column '{p}' not found in pollutant data.")

    # --- Flooring Operation ---

    # Define the columns that represent pollutant levels
    POLLUTANTS_TO_FLOOR = ['PM25', 'O3', 'NO2', 'SO2', 'CO']

    print("--- Flooring Pollutant Levels to 0 ---")

    # Check if the DataFrame exists and is not empty
    if 'df_pollutant' in locals() and not df_pollutant.empty:

        for pollutant in POLLUTANTS_TO_FLOOR:
            if pollutant in df_pollutant.columns:
                # Check for negative values before flooring
                neg_count_before = (df_pollutant[pollutant] < 0).sum()

                # Use clip(lower=0) to replace any value less than 0 with 0
                df_pollutant[pollutant] = df_pollutant[pollutant].clip(lower=0)

                # Check for negative values after flooring
                neg_count_after = (df_pollutant[pollutant] < 0).sum()

                print(f"Processed '{pollutant}': Found and floored {neg_count_before} negative value(s). "
                      f"Negatives remaining: {neg_count_after}.")
            else:
                print(f"Warning: Column '{pollutant}' not found, cannot floor.")

    else:
        print("Please ensure your pollutant data is loaded into a DataFrame named 'df_pollutant'.")


--- 2. Preparing Pollutant Data ---
--- Flooring Pollutant Levels to 0 ---
Processed 'PM25': Found and floored 287 negative value(s). Negatives remaining: 0.
Processed 'O3': Found and floored 1 negative value(s). Negatives remaining: 0.
Processed 'NO2': Found and floored 61 negative value(s). Negatives remaining: 0.
Processed 'SO2': Found and floored 6056 negative value(s). Negatives remaining: 0.
Processed 'CO': Found and floored 1348 negative value(s). Negatives remaining: 0.


In [18]:
# --- 3. Define Seasons ---
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    return None

if 'Month' in df_pollutant.columns:
    df_pollutant['Season'] = df_pollutant['Month'].apply(get_season)

In [19]:
# --- 4. Feature Engineering ---
engineered_features_list = []

if not df_pollutant.empty and 'County Name' in df_pollutant.columns and 'Year' in df_pollutant.columns:
    print("\n--- 4. Engineering Features ---")
    grouped_annual = df_pollutant.groupby(['County Name', 'State Name', 'Year'])

    # --- Define Aggregations for .agg() ---
    aggs = {}

    # Add Latitude and Longitude mean if they exist
    if 'Latitude' in df_pollutant.columns:
        aggs['Latitude'] = 'mean'
    if 'Longitude' in df_pollutant.columns:
        aggs['Longitude'] = 'mean'

    # Add aggregations for each pollutant
    for p in POLLUTANTS:
        if p in df_pollutant.columns:
            aggs[p] = ['mean', 'max', 'std', lambda x: x.quantile(0.75) - x.quantile(0.25)]

    # --- Perform Aggregation ---
    annual_stats = grouped_annual.agg(aggs)

    # Flatten the MultiIndex columns created by .agg()
    annual_stats.columns = ['_'.join(col).strip('_') for col in annual_stats.columns.values]

    # Rename the columns for clarity and consistency
    rename_dict = {
        'Latitude_mean': 'Latitude',
        'Longitude_mean': 'Longitude'
    }
    for p in POLLUTANTS:
        if p in df_pollutant.columns:
            rename_dict[f'{p}_mean'] = f'{p}_Annual_Mean'
            rename_dict[f'{p}_max'] = f'{p}_Annual_Max'
            rename_dict[f'{p}_std'] = f'{p}_Annual_StdDev'
            rename_dict[f'{p}_<lambda>'] = f'{p}_Annual_IQR' # Adjust if lambda name changes

    annual_stats = annual_stats.rename(columns=rename_dict)

    # --- Calculate Days >/< Threshold (Separately) ---
    days_threshold_list = []
    for p in POLLUTANTS:
        if p in THRESHOLDS and p in df_pollutant.columns:
            if p == 'O3':
                days = grouped_annual[p].apply(lambda x: (x < THRESHOLDS[p]).sum()).rename(f'{p}_Days_Below_Threshold')
            else:
                days = grouped_annual[p].apply(lambda x: (x > THRESHOLDS[p]).sum()).rename(f'{p}_Days_Above_Threshold')
            days_threshold_list.append(days)

    # Merge days_threshold data
    if days_threshold_list:
        days_threshold_df = pd.concat(days_threshold_list, axis=1)
        annual_stats = annual_stats.merge(days_threshold_df, on=['County Name', 'State Name', 'Year'], how='left')

    engineered_features_list.append(annual_stats.reset_index())

    # --- Calculate Seasonal Averages (as before) ---
    if 'Season' in df_pollutant.columns:
        grouped_seasonal = df_pollutant.groupby(['County Name', 'State Name', 'Year', 'Season'])
        seasonal_means_list = []
        for p in POLLUTANTS:
            if p in df_pollutant.columns:
                s_mean = grouped_seasonal[p].mean().rename(f'{p}_Seasonal_Avg')
                seasonal_means_list.append(s_mean)

        if seasonal_means_list:
            seasonal_stats_raw = pd.concat(seasonal_means_list, axis=1)
            seasonal_stats_pivot = seasonal_stats_raw.unstack(level='Season')
            seasonal_stats_pivot.columns = ['_'.join(col).strip() for col in seasonal_stats_pivot.columns.values]
            engineered_features_list.append(seasonal_stats_pivot.reset_index())
        else:
            print("No seasonal pollutant data to process.")
    else:
        print("Skipping seasonal features as 'Season' column could not be created.")

# --- 5. Combine Engineered Features ---
df_engineered_features = pd.DataFrame()
if engineered_features_list:
    print("\n--- 5. Combining Engineered Features ---")
    # Start with the first set of features (annual_stats which now includes Lat/Lon)
    df_engineered_features = engineered_features_list[0]
    # Merge subsequent feature sets (e.g., seasonal pivoted data)
    for i in range(1, len(engineered_features_list)):
        df_engineered_features = pd.merge(df_engineered_features, engineered_features_list[i],
                                          on=['County Name', 'State Name', 'Year'], how='outer')
    print(f"Combined engineered features shape: {df_engineered_features.shape}")
    print("Sample of engineered features:")
    print(df_engineered_features.head())
else:
    print("No engineered features were created. Check pollutant data and processing steps.")


--- 4. Engineering Features ---

--- 5. Combining Engineered Features ---
Combined engineered features shape: (341, 50)
Sample of engineered features:
  County Name State Name  Year   Latitude   Longitude  PM25_Annual_Mean  \
0         ada      idaho  2020  43.600699 -116.347853          7.774561   
1         ada      idaho  2021  43.600699 -116.347853          8.714545   
2         ada      idaho  2022  43.600699 -116.347853          7.499138   
3         ada      idaho  2023  43.600699 -116.347853          5.888073   
4         ada      idaho  2024  43.600699 -116.347853          4.489286   

   PM25_Annual_Max  PM25_Annual_StdDev  PM25_<lambda_0>  O3_Annual_Mean  ...  \
0             49.2            8.302082            4.975        0.027381  ...   
1             82.6           10.878881            4.950        0.031094  ...   
2             42.5            6.156418            6.450        0.031984  ...   
3             25.9            4.447445            4.700        0.030272  ... 

In [20]:
# --- 6. Merge with Asthma Data ---
df_final_asthma_gnn = None

if not df_asthma.empty and not df_engineered_features.empty:
    print("\n--- 6. Merging Engineered Features with Asthma Data ---")

    # Standardize 'County Name' in asthma data for merging
    if 'County Name' in df_asthma.columns:
        df_asthma['County Name'] = df_asthma['County Name'].astype(str).str.strip().str.lower()
    else:
        print("ERROR: 'County Name' not in asthma data. Cannot merge.")
        df_asthma = pd.DataFrame() # Make it empty to skip merge

    if 'Year' not in df_asthma.columns:
        print("ERROR: 'Year' not in asthma data. Cannot merge.")
        df_asthma = pd.DataFrame() # Make it empty to skip merge

    # Only attempt merge if required columns are present in asthma data
    if not df_asthma.empty:

        # --- Handle potential column conflicts BEFORE merging ---
        # Get columns from engineered_features, excluding keys to avoid duplication issues
        features_to_add = df_engineered_features.drop(columns=['County Name', 'Year'], errors='ignore')

        # If 'State Name' exists in both, keep the one from engineered_features
        if 'State Name' in df_asthma.columns and 'State Name' in features_to_add.columns:
             df_asthma = df_asthma.drop(columns=['State Name'])

        # If Lat/Lon exist, drop them to prefer Eng_Latitude/Eng_Longitude
        df_asthma = df_asthma.drop(columns=['Latitude', 'Longitude'], errors='ignore')

        # Perform a left merge to keep all asthma records
        df_final_asthma_gnn = pd.merge(df_asthma, df_engineered_features,
                                       on=['County Name', 'Year'],
                                       how='left')

        print(f"Final merged data shape for GNN: {df_final_asthma_gnn.shape}")
        print("Sample of final merged data:")
        print(df_final_asthma_gnn.head())

        # Save this to a new CSV
        output_gnn_data_path = "/content/drive/My Drive/Big Data Project/Data/Processed/asthma_data_with_engineered_pollutants.csv"
        df_final_asthma_gnn.to_csv(output_gnn_data_path, index=False)
        print(f"\nSaved final data to: {output_gnn_data_path}")
    else:
        print("Skipping merge due to missing key columns in asthma data.")

elif df_asthma.empty:
    print("Asthma data is empty. Cannot merge.")

else: # df_engineered_features is empty
    print("No engineered features to merge. The original asthma data remains unchanged.")
    df_final_asthma_gnn = df_asthma

if df_final_asthma_gnn is not None:
    print("\nFinal columns in the dataset for GNN:")
    print(df_final_asthma_gnn.columns.tolist())
else:
    print("\nFinal GNN dataset could not be created.")


--- 6. Merging Engineered Features with Asthma Data ---
Final merged data shape for GNN: (224, 52)
Sample of final merged data:
  County Name  Year  Age-adjusted rate per 10,000  Number of cases  \
0     alameda  2020                         27.32             4274   
1      amador  2020                         28.89              103   
2       butte  2020                         23.98              473   
3   calaveras  2020                         29.14              111   
4      colusa  2020                         25.45               49   

   State Name   Latitude   Longitude  PM25_Annual_Mean  PM25_Annual_Max  \
0  california  37.814781 -122.282347         10.154739       159.708333   
1         NaN        NaN         NaN               NaN              NaN   
2         NaN        NaN         NaN               NaN              NaN   
3         NaN        NaN         NaN               NaN              NaN   
4         NaN        NaN         NaN               NaN              NaN   
